# 8-Queens Problem: Local Search Algorithms

This notebook implements various local search algorithms to solve the 8-Queens problem.  
The goal is to place 8 queens on an N×N chessboard such that no two queens attack each other.

In [1]:
from collections.abc import Iterator
from math import exp
from random import choice, randint, random

N = 8  # Board size

## State Representation

A board state is represented by a string of 8 digits where position $i$ contains the row number of the queen in column $i$.  
Rows are numbered from 1 to 8, starting from the bottom of the board.

**Example**: `'24683157'` represents 8 queens where the queen in column 0 is in row 2, column 1 in row 4, etc.

In [2]:
def heuristic(board: str) -> int:
    """The heuristic cost function h is the number of pairs of queens that are attacking
    each other; this will be zero only for solutions. (It counts as an attack if two
    pieces are in the same line, even if there is an intervening piece between them.)
    """
    h = 0

    for col_i in range(N):
        # Value in column i
        row_i = int(board[col_i])

        for col_j in range(col_i + 1, N):
            # Value in column j
            row_j = int(board[col_j])

            # Same row or same diagonal
            if row_i == row_j or ((col_j - col_i) == abs(row_j - row_i)):
                h += 1

    return h

In [3]:
def random_state() -> str:
    """Generate a random board state by placing one queen in each column at a random row
    position."""
    res = ''

    for _ in range(N):
        res += str(randint(1, N))

    return res


def successors(current: str) -> list[str]:
    """Generate all the successors of a state, i.e. all possible states generated by
    moving a single queen to another square in the same column (so each state has
    8 x 7 = 56 successors)."""
    successors = []

    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                successors.append(current[:col] + str(new_row) + current[col + 1 :])

    return successors


def successors_generator(current: str) -> Iterator[str]:
    """Generate successor states lazily using a generator.

    Memory-efficient alternative to successors() that yields states one at a time
    instead of creating the entire list in memory.
    """
    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                yield current[:col] + str(new_row) + current[col + 1 :]


def random_successor(current: str) -> str:
    """Generate a random successor of the current state."""
    col = randint(0, N - 1)
    current_row = int(current[col])

    new_row = randint(1, N)
    # If the value is the same as the current one, try again
    while new_row == current_row:
        new_row = randint(1, N)

    return current[:col] + str(new_row) + current[col + 1 :]

In [4]:
def display(board: str) -> None:
    """Display the board state in a visual format."""
    grid = [['_'] * N for _ in range(N)]

    for col, row_char in enumerate(board):
        row = int(row_char)
        grid[N - row][col] = 'Q'

    for row in grid:
        print(' '.join(row))
    print(f'Conflicts: {heuristic(board)}\n')

## Hill Climbing Algorithms

In [5]:
def hill_climbing(initial_state: str) -> tuple[str, dict[str, int | bool]]:
    """Steepest-ascent hill climbing algorithm."""
    current = initial_state
    h_current = heuristic(initial_state)
    steps = 0

    while True:
        # Find best successor
        children = [
            (successor, heuristic(successor)) for successor in successors(current)
        ]
        neighbor, h_neighbor = min(children, key=lambda x: x[1])
        steps += 1

        # Stop if no improvement
        if h_neighbor >= h_current:
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats

        current, h_current = neighbor, h_neighbor


def hill_climbing_sideways_move(
    initial_state: str,
) -> tuple[str, dict[str, int | bool]]:
    """Hill climbing with sideways moves to escape local shoulders."""
    current = initial_state
    h_current = heuristic(initial_state)
    sideways_move = 0
    steps = 0

    while True:
        # Generate all successors
        children = [
            (successor, heuristic(successor)) for successor in successors(current)
        ]

        # Filter on the best heuristic value
        h_min = min(h for _, h in children)
        best_children = [(s, h) for s, h in children if h == h_min]

        # We choose a random neighbor among the best ones
        neighbor, h_neighbor = choice(best_children)
        steps += 1

        # Stop if no improvement or limit of sideways move is reached
        if (h_neighbor > h_current) or (sideways_move >= 100):
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats

        # Update sideway value
        if h_neighbor == h_current:
            sideways_move += 1
        else:
            sideways_move = 0

        current, h_current = neighbor, h_neighbor


def first_choice_hill_climbing(initial_state: str) -> tuple[str, dict[str, int | bool]]:
    """First-choice hill climbing algorithm.

    Generates successors and accepts the first one that improves the current state.
    Useful when there are many successors."""
    current = initial_state
    h_current = heuristic(initial_state)
    steps = 0

    while True:
        for child in successors_generator(current):
            h_child = heuristic(child)

            if h_child < h_current:
                current, h_current = child, h_child
                steps += 1
                break

        # Finally found the use of an 'else' on a 'for' loop, fuck yeah!
        # If we have generated all successors without finding a better one
        else:
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats


def random_restart_hill_climbing() -> tuple[str, dict[str, int | bool]]:
    """Hill climbing with random restarts.

    Repeatedly runs hill climbing from random initial states until a solution
    is found. Guarantees finding a solution eventually (with probability 1)."""
    restarts = 0
    total_steps = 0

    while True:
        current_state = random_state()
        result, stats = hill_climbing(current_state)
        total_steps += stats['steps']

        # Success
        if stats['success']:
            stats['restarts'] = restarts
            stats['total_steps'] = total_steps
            return result, stats

        restarts += 1


def random_restart_hill_climbing_sideways_move() -> tuple[str, dict[str, int | bool]]:
    """Hill climbing with sideways moves and random restarts."""
    restarts = 0
    total_steps = 0

    while True:
        current_state = random_state()
        result, stats = hill_climbing_sideways_move(current_state)
        total_steps += stats['steps']

        # Success
        if stats['success']:
            stats['restarts'] = restarts
            stats['total_steps'] = total_steps
            return result, stats

        restarts += 1

## Simulated annealing

In [6]:
def simulated_annealing(
    initial_state: str,
    max_iterations: int = 1000,
    initial_temp: int = 100,
    alpha: float = 0.90,
) -> tuple[str, dict[str, int | bool]]:
    """Simulated annealing with exponential cooling."""
    current = initial_state
    h_current = heuristic(initial_state)
    temperature = initial_temp
    steps = 0

    for iterations in range(max_iterations):
        # Success
        if h_current == 0:
            break

        child = random_successor(current)
        h_child = heuristic(child)
        delta_e = h_current - h_child

        # If the move improves the situation (i.e delta_e > 0), it is always accepted
        # Otherwise if the move is worse (delta_e < 0),
        # the algorithm accepts the move with some probability less than 1
        if (delta_e > 0) or (exp(delta_e / temperature) > random()):
            current, h_current = child, h_child
            steps += 1

        # Cool down
        temperature *= alpha

    stats = {
        'steps': steps,
        'iterations': iterations,
        'h_final': h_current,
        'success': h_current == 0,
    }
    return current, stats

### Local Beam Search

In [7]:
def local_beam_search(
    k_size: int, max_steps: int = 40
) -> tuple[str, dict[str, int | bool]]:
    """Local beam search algorithm

    Keeps track of k states rather than just one. It begins with k randomly generated states.
    At each step, all the successors of all k states are generated.
    If any one is a goal, the algorithm halts.
    Otherwise, it selects the k best successors from the complete list and repeats."""
    # Initialize k random states
    current_states = [random_state() for _ in range(k_size)]
    best_h = 1000
    no_improvement_count = 0
    steps = 0

    # Check if any initial state is a goal
    for state in current_states:
        if heuristic(state) == 0:
            return state, {'steps': 0, 'h_final': 0, 'success': True}

    # Make sure we don't loop forever
    while steps < max_steps:
        # Generate all successors
        children = [
            (successor, heuristic(successor))
            for current in current_states
            for successor in successors(current)
        ]

        # Sort by heuristic
        children.sort(key=lambda x: x[1])
        current_best_state, current_best_h = children[0]
        steps += 1

        # Success
        if current_best_h == 0:
            stats = {'steps': steps, 'h_final': 0, 'success': True}
            return current_best_state, stats

        if current_best_h < best_h:
            best_h = current_best_h
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Early stopping if no improvement
        if no_improvement_count >= 10:
            break

        # We keep the k best successors
        current_states = [child[0] for child in children[:k_size]]

    stats = {'steps': steps, 'h_final': heuristic(current_states[0]), 'success': False}
    return current_states[0], stats

## From a specific state :

In [8]:
s = '43254323'
display(s)
print(s)

_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ Q _ _ _ _
Q _ _ _ Q _ _ _
_ Q _ _ _ Q _ Q
_ _ Q _ _ _ Q _
_ _ _ _ _ _ _ _
Conflicts: 17

43254323


### Hill Climbing

In [9]:
print('>>> Hill Climbing:')
res, stats = hill_climbing(s)
display(res)
print(res)
stats

>>> Hill Climbing:
_ _ _ _ _ _ _ _
_ _ _ _ _ _ Q _
_ Q _ _ _ _ _ _
_ _ _ Q _ _ _ _
_ _ _ _ _ Q _ _
Q _ _ _ _ _ _ Q
_ _ Q _ _ _ _ _
_ _ _ _ Q _ _ _
Conflicts: 1

36251473


{'steps': 6, 'h_final': 1, 'success': False}

In [10]:
print('>>> Hill Climbing Sideways Move:')
res, stats = hill_climbing_sideways_move(s)
display(res)
print(res)
stats

>>> Hill Climbing Sideways Move:
_ _ _ _ Q _ _ _
Q _ _ _ _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ Q _ _ _ _
_ Q _ _ _ _ _ _
_ _ _ _ _ _ Q _
_ _ Q _ _ _ _ _
_ _ _ _ _ Q _ _
Conflicts: 0

74258136


{'steps': 38, 'h_final': 0, 'success': True}

In [11]:
print('>>> First Choice Hill Climbing:')
res, stats = first_choice_hill_climbing(s)
display(res)
print(res)
stats

>>> First Choice Hill Climbing:
_ Q _ _ _ _ _ _
_ _ _ _ _ Q _ _
Q _ _ _ _ _ _ _
_ _ Q _ _ _ _ _
_ _ _ Q _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ _ _ _ Q _
_ _ _ _ Q _ _ _
Conflicts: 2

68541723


{'steps': 11, 'h_final': 2, 'success': False}

### Simulated Annealing

In [12]:
print('>>> Simulated Annealing:')
res, stats = simulated_annealing(s)
display(res)
print(res)
stats

>>> Simulated Annealing:
_ _ Q _ _ _ _ _
_ _ _ _ Q _ _ _
_ Q _ _ _ _ _ _
_ _ _ _ _ _ _ Q
Q _ _ _ _ _ _ _
_ _ _ _ _ _ Q _
_ _ _ Q _ _ _ _
_ _ _ _ _ Q _ _
Conflicts: 0

46827135


{'steps': 57, 'iterations': 127, 'h_final': 0, 'success': True}

## From a random state :

In [13]:
r = random_state()
display(r)
print(r)

_ _ Q _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ Q _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ Q _
Q Q _ _ Q Q _ _
Conflicts: 8

11851126


### Hill Climbing

In [14]:
print('>>> Hill Climbing:')
res, stats = hill_climbing(r)
display(res)
print(res)
stats

>>> Hill Climbing:
_ _ Q _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ Q _ _ _ _
_ _ _ _ _ Q _ _
Q _ _ _ _ _ _ _
_ _ _ _ _ _ Q _
_ Q _ _ Q _ _ _
Conflicts: 3

31851426


{'steps': 3, 'h_final': 3, 'success': False}

In [15]:
print('>>> Hill Climbing Sideways Move:')
res, stats = hill_climbing_sideways_move(r)
display(res)
print(res)
stats

>>> Hill Climbing Sideways Move:
_ _ Q _ _ _ _ _
_ _ _ _ Q _ _ _
_ _ _ _ _ _ _ Q
_ _ _ Q _ _ _ _
Q _ _ _ _ _ _ _
_ _ _ _ _ _ Q _
_ Q _ _ _ _ _ _
_ _ _ _ _ Q _ _
Conflicts: 0

42857136


{'steps': 5, 'h_final': 0, 'success': True}

In [16]:
print('>>> First choice Hill Climbing:')
res, stats = first_choice_hill_climbing(r)
display(res)
print(res)
stats

>>> First choice Hill Climbing:
_ _ Q _ _ _ _ _
_ _ _ _ _ _ _ _
_ Q _ _ _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ _ _ Q _ _
Q _ _ Q _ _ _ _
_ _ _ _ _ _ Q _
_ _ _ _ Q _ _ _
Conflicts: 1

36831425


{'steps': 5, 'h_final': 1, 'success': False}

### Simulated Annealing

In [17]:
print('>>> Simulated Annealing:')
res, stats = simulated_annealing(r)
display(res)
print(res)
stats

>>> Simulated Annealing:
_ _ _ _ _ Q _ _
Q _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ Q _ _ _
_ _ _ _ _ _ _ Q
_ Q _ Q _ _ _ _
_ _ _ _ _ _ Q _
_ _ Q _ _ _ _ _
Conflicts: 1

73135824


{'steps': 112, 'iterations': 999, 'h_final': 1, 'success': False}

In [ ]:
# %timeit hill_climbing(r)
# %timeit hill_climbing_sideways_move(r)
# %timeit first_choice_hill_climbing(r)
# %timeit simulated_annealing(r)

1.84 ms ± 659 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.06 ms ± 469 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.71 ms ± 140 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
6.3 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Random restart algorithms :

In [19]:
print('>>> Random restart Hill Climbing:')
res, stats = random_restart_hill_climbing()
display(res)
print(res)
stats

>>> Random restart Hill Climbing:
_ _ Q _ _ _ _ _
_ _ _ _ _ Q _ _
_ _ _ Q _ _ _ _
Q _ _ _ _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ _ Q _ _ _
_ _ _ _ _ _ Q _
_ Q _ _ _ _ _ _
Conflicts: 0

51863724


{'steps': 5, 'h_final': 0, 'success': True, 'restarts': 4, 'total_steps': 22}

In [20]:
print('>>> Random restart Hill Climbing Sideways Move:')
res, stats = random_restart_hill_climbing_sideways_move()
display(res)
print(res)
stats

>>> Random restart Hill Climbing Sideways Move:
Q _ _ _ _ _ _ _
_ _ _ _ Q _ _ _
_ _ _ _ _ _ _ Q
_ _ _ _ _ Q _ _
_ _ Q _ _ _ _ _
_ _ _ _ _ _ Q _
_ Q _ _ _ _ _ _
_ _ _ Q _ _ _ _
Conflicts: 0

82417536


{'steps': 17, 'h_final': 0, 'success': True, 'restarts': 0, 'total_steps': 17}

## Local Beam Search

In [21]:
print('>>> Local Beam Search:')
res, stats = local_beam_search(k_size=20)
display(res)
print(res)
stats

>>> Local Beam Search:
_ _ Q _ _ _ _ _
_ _ _ _ _ _ _ Q
_ _ _ Q _ _ _ _
_ _ _ _ _ _ Q _
Q _ _ _ _ _ _ _
_ _ _ _ _ Q _ _
_ Q _ _ _ _ _ _
_ _ _ _ Q _ _ _
Conflicts: 0

42861357


{'steps': 8, 'h_final': 0, 'success': True}

In [22]:
sum([local_beam_search(k_size=20)[1]['success'] for _ in range(100)])

96

In [ ]:
# %timeit random_restart_hill_climbing()
# %timeit random_restart_hill_climbing_sideways_move()
# %timeit local_beam_search(k_size=20)

20 ms ± 8.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.7 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
44.5 ms ± 5.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing())
# len(res)
# # res

In [25]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing_sideways_move())
# len(res)
# # res